In [32]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objs as go

# Descargar los datos de QQQ
data = yf.download("BTC-USD", start="2023-12-01", end="2024-04-01",interval='1d')

# Obtener los rendimientos diarios
returns = data['Adj Close'].pct_change().dropna()

# Configurar las simulaciones
n_simulations = 100
n_days = len(returns)

# Realizar las simulaciones de bootstrap
bootstrap_samples = np.random.choice(returns, (n_simulations, n_days), replace=True)
bootstrap_cumulated = np.cumprod(1 + bootstrap_samples, axis=1)

# Calcular el precio simulado para cada muestra de bootstrap
simulated_prices = bootstrap_cumulated * data['Adj Close'].iloc[0]

# Crear un DataFrame para las simulaciones
simulated_df = pd.DataFrame(simulated_prices.T)

# Graficar las simulaciones
fig = go.Figure()

# Agregar líneas para cada simulación
for i in range(n_simulations):
    fig.add_trace(go.Scatter(x=data.index, y=simulated_df[i], mode='lines', line=dict(width=2), opacity=0.3))

# Agregar línea del precio real
fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Real', line=dict(color='black', width=2)))

# Configurar el gráfico
fig.update_layout(title='Simulaciones de Bootstrap para BTS-USD',
                  xaxis_title='Fecha',
                  yaxis_title='Precio Ajustado',
                  template='plotly_white')

# Mostrar el gráfico
fig.show()


[*********************100%%**********************]  1 of 1 completed


In [14]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from scipy.optimize import minimize

# Descargar los datos de QQQ
data = yf.download("QQQ", start="2023-01-01", end="2023-12-31")

# Obtener los rendimientos diarios
returns = data['Adj Close'].pct_change().dropna()

# Identificar eventos: días con rendimientos superiores al cuantil 95%
event_times = returns.index[returns > returns.quantile(0.95)].astype(np.int64) / 1e9  # Convertir fechas a segundos

# Definir la función de intensidad de un proceso de Hawkes
def hawkes_intensity(t, events, mu, alpha, beta):
    intensity = mu
    for event_time in events:
        if event_time < t:
            intensity += alpha * np.exp(-beta * (t - event_time))
    return intensity

# Función de log-verosimilitud negativa
def neg_log_likelihood(params, events):
    mu, alpha, beta = params
    log_likelihood = 0.0
    for i in range(1, len(events)):
        dt = events[i] - events[i-1]
        intensity = hawkes_intensity(events[i], events[:i], mu, alpha, beta)
        log_likelihood += np.log(intensity) - intensity * dt
    return -log_likelihood

# Estimar los parámetros del proceso de Hawkes usando los datos reales
initial_params = [0.1, 0.5, 1.0]  # Valores iniciales para mu, alpha, beta

# Optimizar la log-verosimilitud negativa
result = minimize(neg_log_likelihood, initial_params, args=(event_times,), bounds=[(1e-5, None), (1e-5, None), (1e-5, None)])
mu_est, alpha_est, beta_est = result.x

# Simular el proceso de Hawkes
def simulate_hawkes(T, mu, alpha, beta):
    t = 0
    events = []
    while t < T:
        intensity = hawkes_intensity(t, events, mu, alpha, beta)
        wait_time = np.random.exponential(1.0 / intensity)
        t += wait_time
        if t < T:
            events.append(t)
    return np.array(events)

# Predicción de eventos futuros para el próximo año
future_events = simulate_hawkes(365 * 24 * 60 * 60, mu_est, alpha_est, beta_est)  # Simular un año en segundos
future_dates = pd.to_datetime(future_events, unit='s', origin=data.index[-1])

# Graficar la intensidad y eventos predichos
fig = go.Figure()

# Intensidad calculada a partir de los datos reales
intensity_values = [hawkes_intensity(t, event_times, mu_est, alpha_est, beta_est) for t in event_times]

# Graficar la intensidad del proceso
fig.add_trace(go.Scatter(x=data.index, y=intensity_values, mode='lines', name='Intensidad', line=dict(color='blue')))

# Graficar eventos futuros
fig.add_trace(go.Scatter(x=future_dates, y=np.ones_like(future_events), mode='markers', name='Eventos Futuros', marker=dict(color='red', size=5)))

# Configuración del gráfico
fig.update_layout(title='Modelo de Proceso de Hawkes para QQQ',
                  xaxis_title='Fecha',
                  yaxis_title='Intensidad',
                  template='plotly_white')

# Mostrar el gráfico
fig.show()


[*********************100%%**********************]  1 of 1 completed


In [16]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from scipy.optimize import minimize

# Descargar los datos de QQQ
data = yf.download("QQQ", start="2021-01-01", end="2023-12-31")

# Obtener los rendimientos diarios
returns = data['Adj Close'].pct_change().dropna()

# Identificar eventos: días con rendimientos superiores al cuantil 95%
event_times = returns.index[returns > returns.quantile(0.95)].astype(np.int64) / 1e9  # Convertir fechas a segundos

# Definir la función de intensidad de un proceso de Hawkes
def hawkes_intensity(t, events, mu, alpha, beta):
    intensity = mu
    for event_time in events:
        if event_time < t:
            intensity += alpha * np.exp(-beta * (t - event_time))
    return intensity

# Función de log-verosimilitud negativa
def neg_log_likelihood(params, events):
    mu, alpha, beta = params
    log_likelihood = 0.0
    for i in range(1, len(events)):
        dt = events[i] - events[i-1]
        intensity = hawkes_intensity(events[i], events[:i], mu, alpha, beta)
        log_likelihood += np.log(intensity) - intensity * dt
    return -log_likelihood

# Estimar los parámetros del proceso de Hawkes usando los datos reales
initial_params = [0.1, 0.5, 1.0]  # Valores iniciales para mu, alpha, beta

# Optimizar la log-verosimilitud negativa
result = minimize(neg_log_likelihood, initial_params, args=(event_times,), bounds=[(1e-5, None), (1e-5, None), (1e-5, None)])
mu_est, alpha_est, beta_est = result.x

# Simular el proceso de Hawkes
def simulate_hawkes(T, mu, alpha, beta):
    t = 0
    events = []
    while t < T:
        intensity = hawkes_intensity(t, events, mu, alpha, beta)
        wait_time = np.random.exponential(1.0 / intensity)
        t += wait_time
        if t < T:
            events.append(t)
    return np.array(events)

# Predicción de eventos futuros para el próximo año
future_events = simulate_hawkes(365 * 24 * 60 * 60, mu_est, alpha_est, beta_est)  # Simular un año en segundos
future_dates = pd.to_datetime(future_events, unit='s', origin=data.index[-1])

# Intensidad calculada a partir de los datos reales
intensity_values = [hawkes_intensity(t, event_times, mu_est, alpha_est, beta_est) for t in event_times]

# 1. Histogram of Event Intervals
event_intervals = np.diff(event_times)
fig1 = px.histogram(event_intervals, nbins=30, title='Histogram of Event Intervals',
                    labels={'value': 'Time Between Events (seconds)'}, template='plotly_white')

# 2. Intensity Heatmap
heatmap_data = pd.DataFrame({
    'Time': returns.index[1:],  # Skip the first index because of .pct_change()
    'Intensity': [hawkes_intensity(t, event_times, mu_est, alpha_est, beta_est) for t in returns.index[1:].astype(np.int64) / 1e9]
})
fig2 = px.density_heatmap(heatmap_data, x='Time', y='Intensity', title='Intensity Heatmap Over Time', template='plotly_white')

# 3. Cumulative Intensity Plot
cumulative_intensity = np.cumsum(intensity_values)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=data.index, y=cumulative_intensity, mode='lines', name='Cumulative Intensity', line=dict(color='purple')))
fig3.update_layout(title='Cumulative Intensity of Hawkes Process',
                   xaxis_title='Date', yaxis_title='Cumulative Intensity', template='plotly_white')

# 4. Overlay of Simulated and Real Events
fig4 = go.Figure()
# Real events
fig4.add_trace(go.Scatter(x=pd.to_datetime(event_times, unit='s'), y=np.ones_like(event_times), mode='markers',
                          name='Real Events', marker=dict(color='green', size=7)))
# Simulated events
fig4.add_trace(go.Scatter(x=future_dates, y=np.ones_like(future_events) * 1.05, mode='markers',
                          name='Simulated Events', marker=dict(color='orange', size=5)))
fig4.update_layout(title='Overlay of Simulated and Real Events',
                   xaxis_title='Date', yaxis_title='Event Occurrence', template='plotly_white')

# Display all plots
fig1.show()
fig2.show()
fig3.show()
fig4.show()


[*********************100%%**********************]  1 of 1 completed
